<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# Mathematics Basics

**With `pandas`**

See also `45_math_basics.ipynb`.

&copy; Dr. Yves J. Hilpisch | The Python Quants GmbH

http://tpq.io | [training@tpq.io](mailto:trainin@tpq.io) | [@dyjh](http://twitter.com/dyjh)

In [ ]:
!git clone https://github.com/tpq-classes/mathematics_basics.git
import sys
sys.path.append('mathematics_basics')


In [ ]:
import numpy as np
import pandas as pd
from numpy.random import default_rng

In [ ]:
np.set_printoptions(suppress=True)
pd.set_option('display.float_format', lambda x: '%.4f' % x)

## Time Series Data

In [ ]:
rng = default_rng(1000)

In [ ]:
rn = rng.standard_normal((365, 3))

In [ ]:
rn[0] = 3 * [0.]

In [ ]:
rn

In [ ]:
index = pd.date_range('2021-1-1', '2021-12-31', freq='D')

In [ ]:
index[:10]

In [ ]:
df = pd.DataFrame(rn.cumsum(axis=0), index=index)

In [ ]:
df

In [ ]:
from pylab import plt
plt.style.use('seaborn-v0_8')
%config InlineBackend.figure_format = 'svg'

In [ ]:
df.plot();

## Indexing & Selection

In [ ]:
df[0]

In [ ]:
df.loc['2021-01-01']

In [ ]:
df[1]

In [ ]:
df.iloc[1]

In [ ]:
df.loc['2021-01-02']

In [ ]:
df[0:4]

In [ ]:
df.loc['2021-01-01':'2021-01-04']

In [ ]:
df[:4]

In [ ]:
df.loc[:'2021-01-04']

In [ ]:
df[3:6]

In [ ]:
df.loc['2021-01-04':'2021-01-06']

In [ ]:
df[5:]

In [ ]:
df.loc['2021-01-05':]

## Vectorized Arithmetics

In [ ]:
a = 1 / 2

In [ ]:
a * df

In [ ]:
df + df

In [ ]:
df * df

## Statistics

In [ ]:
df.sum()

In [ ]:
df.mean()

In [ ]:
df.var()

In [ ]:
df.std()

In [ ]:
df.min()

In [ ]:
df.max()

In [ ]:
df.median()

In [ ]:
df.median(axis=0)

In [ ]:
df.sum(axis=1)

In [ ]:
df.mean(axis=1)

In [ ]:
df.var(axis=1)

## Rolling Statistics

In [ ]:
window = 20

In [ ]:
r = df.rolling(window).mean()

In [ ]:
r

In [ ]:
# r.head(20)

In [ ]:
ax = df.plot(legend=False, cmap='coolwarm')
r.plot(label='SMA', legend=False, ax=ax);

In [ ]:
r = df.rolling(window).median()

In [ ]:
r

In [ ]:
ax = df.plot(legend=False, cmap='coolwarm')
r.plot(label='MEDIAN', legend=False, ax=ax);

In [ ]:
r = df.rolling(window).min()

In [ ]:
r

In [ ]:
ax = df.plot(legend=False, cmap='coolwarm')
r.plot(label='MIN', legend=False, ax=ax);

In [ ]:
r = df.rolling(window).max()

In [ ]:
r

In [ ]:
ax = df.plot(legend=False, cmap='coolwarm')
r.plot(label='MAX', legend=False, ax=ax);

In [ ]:
r = df.rolling(window).std()

In [ ]:
r

In [ ]:
# df.plot(legend=False)
r.plot(label='STD', legend=False);

In [ ]:
r = df.ewm(halflife=5).mean()  # no minimum periods

In [ ]:
r

In [ ]:
r = df.ewm(halflife=5, min_periods=window).mean()  # with minimum periods

In [ ]:
r

In [ ]:
ax = df.plot(legend=False, cmap='coolwarm')
r.plot(label='EWMA', legend=False, ax=ax);

## Custom Functions

In [ ]:
N = int(1e6)

In [ ]:
a = int(N / 10)
a

In [ ]:
b = int(N / a)

In [ ]:
b

In [ ]:
%time df = pd.DataFrame(rng.standard_normal(N).reshape(a, b).cumsum(axis=0))

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
df.plot(lw=1, cmap='coolwarm', legend=False);

### Element-Wise Functions

In [ ]:
import math

In [ ]:
# df.apply(math.cos)  # does not work

In [ ]:
np.cos(df)

In [ ]:
df.apply(np.cos)

In [ ]:
%timeit df.apply(np.cos)

In [ ]:
df / 2

In [ ]:
%timeit df / 2

In [ ]:
def f(x):
    return x / 2

In [ ]:
df.apply(f)

In [ ]:
%timeit df.apply(f)

In [ ]:
df.apply(lambda x: x / 2)

In [ ]:
%timeit df.apply(lambda x: x / 2)

### `numexpr` Module

In [ ]:
np.sqrt(np.abs(df)) + np.sin(df)

In [ ]:
%timeit np.sqrt(np.abs(df)) + np.sin(df)

In [ ]:
pd.eval('sqrt(abs(df)) + sin(df)')

In [ ]:
%timeit pd.eval('sqrt(abs(df)) + sin(df)')

### Recursive Functions

In [ ]:
def ewma_py(x, alpha=0.25):
    y = np.zeros_like(x)
    y[0] = x[0]
    for i in range(1, len(x)):
        y[i] = alpha * x[i] + (1 - alpha) * y[i - 1]
    return y

In [ ]:
a = ewma_py(df[0])

In [ ]:
a

In [ ]:
r_py = pd.Series(ewma_py(df[0]))

In [ ]:
r_py

In [ ]:
df[0].iloc[-100:].plot()
r_py.iloc[-100:].plot();

In [ ]:
%timeit r_py = pd.Series(ewma_py(df[0]))

In [ ]:
import numba

In [ ]:
ewma_nb = numba.jit(ewma_py)

In [ ]:
df[0].values  # transforming Series object to ndarray object

In [ ]:
r_nb = pd.Series(ewma_nb(df[0].values))

In [ ]:
r_nb

In [ ]:
(r_py == r_nb).all()  # real equality

In [ ]:
np.allclose(r_py, r_nb)  # equality to a certain precision level

In [ ]:
%timeit r_nb = pd.Series(ewma_nb(df[0].values))

## Time Series Data

In [ ]:
url = 'http://hilpisch.com/aiif_eikon_eod_data.csv'

In [ ]:
data = pd.read_csv(url, index_col=0, parse_dates=True).dropna()

In [ ]:
type(data)

In [ ]:
data

In [ ]:
data.index

In [ ]:
data.info()

In [ ]:
data.plot();

In [ ]:
(data / data.iloc[0]).plot();

In [ ]:
sym = 'AAPL.O'

In [ ]:
data[sym].plot()
data[sym].rolling(42).mean().plot(ls='--')
data[sym].rolling(252).mean().plot(style='-.');

In [ ]:
data[sym].plot(lw=1)
data[sym].rolling(50).min().plot(ls='--', lw=1)
data[sym].rolling(50).max().plot(style='-.', lw=1);

## Exporting Data

In [ ]:
data.to_numpy()  # like: data.values

In [ ]:
data.to_records()

In [ ]:
d = data.to_dict()

In [ ]:
# d

In [ ]:
d[sym][pd.Timestamp('2010-01-07')]

In [ ]:
for key in list(d[sym].keys())[:5]:
    print(d[sym][key])

In [ ]:
data.to_csv()[:300]

In [ ]:
data.to_html()[:500]

In [ ]:
data.to_markdown()[:500]

In [ ]:
data.to_latex()[:500]

In [ ]:
data.to_excel('data.xlsx')

In [ ]:
index = [str(t)[:10] for t in data.index]

In [ ]:
index[:6]

In [ ]:
data.index = index

In [ ]:
data.to_json('data.json')

## <img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:training@tpq.io">training@tpq.io</a>